Diff PRE-POST

In [ ]:
from scipy.stats import gaussian_kde

def diff_size(nonPicked_ratio, all_measures, nonPicked_measures):
    picked_ratio = 1 - nonPicked_ratio

    # By miu,sigma
    miu_all = all_measures.mean()
    sigma_all = all_measures.std()
    miu_nonPicked = nonPicked_measures.mean()
    sigma_nonPicked = nonPicked_measures.std()
    miu_picked = 1 / picked_ratio * miu_all - (nonPicked_ratio / picked_ratio) * miu_nonPicked
    sigma_picked = np.sqrt((1 / picked_ratio) ** 2 * sigma_all ** 2 - (nonPicked_ratio / picked_ratio) ** 2 * sigma_nonPicked ** 2)

    # By kde
    kde_all = gaussian_kde(all_measures)
    kde_nonPicked = gaussian_kde(nonPicked_measures)
    kde_picked = 1 / picked_ratio * kde_all - (nonPicked_ratio / picked_ratio) * kde_nonPicked
    return (miu_picked, sigma_picked), kde_picked

Debug

In [29]:
import os
os.chdir('/home/yotam/FruitSpec/Code/fsCounter')
print(os.getcwd())
# from vision.data.results_collector import ResultsCollector
from vision.tools.manual_slicer import slice_to_trees
import math
import pandas as pd

/home/yotam/FruitSpec/Code/fsCounter


In [30]:
def filter_logic(path):
    df = pd.read_csv(os.path.join(path,'measures.csv'))
    df_res = []
    ids = df.groupby(['track_id'])
    for _, df_id in ids:
        # if len(df_id) < 4:
        #     continue
        df_res.append(df_id)
    df_res = pd.concat(df_res)
    return df_res

In [31]:
def trackers_into_plot(df_tree, df_res):
    df_tree['end'].replace([-1], math.inf,inplace=True)
    plot_det = []
    for i, df_frame in df_res.groupby(['frame']):
        frame_id = df_frame.iloc[0]['frame']
        limit = df_tree[df_tree['frame_id'] == frame_id]
        if limit.empty:
            continue
        limit = limit.iloc[0]
        df = df_frame[(df_frame['x1'] > limit['start']) & (df_frame['x1'] < limit['end'])]
        plot_det.append(df)

    fruits = len(pd.concat(plot_det,axis=0)['track_id'].unique())
    return fruits

In [32]:
def run(scan,output):
    df_sum = pd.DataFrame()
    for row in os.listdir(scan):
        row_folder = os.path.join(scan,row)
        if os.path.isfile(row_folder):
            continue
        df_res = filter_logic(row_folder)
        trees_slices = slice_to_trees(os.path.join(row_folder, [i for i in os.listdir(row_folder) if 'slice_data' in i][0]),None,None)
        fruits_num = trackers_into_plot(trees_slices,df_res)
        df_sum = df_sum.append({'repeat':row,'fruits_num':fruits_num},ignore_index = True)
    df_sum.to_csv(output,index=True,header=True,mode='w')
    return df_sum

In [41]:
scan_folder = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222'
output_path = fr'/media/yotam/Extreme SSD/syngenta trail/tomato/analysis/281222/results.csv'
DF = run(scan_folder,output_path)

/tmp/ipykernel_535546/361793406.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, df_id in ids:
/tmp/ipykernel_535546/1561645460.py:4: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df_frame in df_res.groupby(['frame']):
/tmp/ipykernel_535546/2163329521.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_sum = df_sum.append({'repeat':row,'fruits_num':fruits_num},ignore_index = True)
/tmp/ipykernel_535546/361793406.py:5: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a gr

Color debug

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np

def plot_bbox(data, index):
    f = int(data['frame'])
    x1 = int(data['x1'])
    y1 = int(data['y1'])
    x2 = int(data['x2'])
    y2 = int(data['y2'])
    if x1 < 0 or x2 < 0 or y1 < 0 or y2 < 0:
        return
    im = cv2.imread(os.path.join(r'E:\syngenta trail\tomato\analysis\061022\YATED61R\frames', f'frame_{f}_res.jpg'))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    # im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE)

    crop = im[y1:y2, x1:x2, :].copy()
    # cv2.imwrite(os.path.join(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\081122\4\hue', f'{index[1]}_RGB.jpg'),crop)
    return crop


def get_color(det_crop, index=None, saturation_threshold=25, precentile=0.5):
    hsv = cv2.cvtColor(det_crop, cv2.COLOR_RGB2HSV)
    hue = hsv[:, :, 0].copy()
    hue *= 2
    h, b = np.histogram(hue[hsv[:, :, 1] > saturation_threshold].flatten(), 360)
    p = np.cumsum(h) / sum(h)

    for i, percent in enumerate(p):
        if percent > precentile:
            break

    fig, (ax1,ax2) = plt.subplots(1,2)
    ax1.plot(h)
    ax2.imshow(det_crop)
    fig.savefig(rf'E:\syngenta trail\tomato\analysis\061022\YATED61R\HUE_hist\2676_frame_{index}.jpg')
    plt.close()
    return round(b[i]*2, 1),round(b.mean()*2, 1)

In [ ]:
df = pd.read_csv(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\061022\color_for_hist_2676.csv')
df['color_median'] = None
df['color_mean'] = None
for j, row in df.iterrows():
    try:
        hue_med,hue_mean = get_color(plot_bbox(row, (0, j)), index=row['frame'])
        df.at[j, 'color_median'] = hue_med
        df.at[j, 'color_mean'] = hue_mean
    except Exception as e:
        print(repr(e))
df.to_csv(r'C:\Users\frodo\OneDrive - FruitSpec\Syngenta\061022\color_for_hist_2676.csv', index=False)